In [ ]:
import socket
import threading

# Konfigurasi server
HOST = '127.0.0.1'   # menerima koneksi dari semua IP
PORT = 8080

clients = []
lock = threading.Lock()

def broadcast(message, sender_socket=None):
    """Kirim pesan ke semua client kecuali pengirim."""
    with lock:
        disconnected = []
        for client in clients:
            if client == sender_socket:
                continue
            try:
                client.send(message)
            except Exception as e:
                print(f"[ERROR] Tidak bisa kirim ke client: {e}")
                disconnected.append(client)
        # Hapus client yang sudah terputus
        for dc in disconnected:
            if dc in clients:
                clients.remove(dc)
                dc.close()

def handle_client(client_socket, address):
    """Menangani komunikasi dengan satu client."""
    print(f"[TERHUBUNG] {address}")
    try:
        while True:
            msg = client_socket.recv(1024)
            if not msg:
                break  # client keluar
            decoded_msg = msg.decode().strip()
            print(f"[{address}] {decoded_msg}")
            # Kirim pesan ke semua client lain
            broadcast(f"[{address}] {decoded_msg}".encode(), client_socket)
    except (ConnectionResetError, ConnectionAbortedError):
        print(f"[DISKONEK] Client {address} terputus secara tiba-tiba.")
    finally:
        with lock:
            if client_socket in clients:
                clients.remove(client_socket)
        client_socket.close()
        print(f"[PUTUS] {address} keluar dari server.")

def start_server():
    """Menjalankan server dan menerima koneksi baru."""
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # hindari error port in use
    server.bind((HOST, PORT))
    server.listen(10)
    print(f"[SERVER AKTIF] Menunggu koneksi di {HOST}:{PORT}")

    try:
        while True:
            client_socket, addr = server.accept()
            with lock:
                clients.append(client_socket)
            thread = threading.Thread(target=handle_client, args=(client_socket, addr), daemon=True)
            thread.start()
    except KeyboardInterrupt:
        print("\n[SERVER DsIMATIKAN OLEH USER]")
    finally:
        with lock:
            for c in clients:
                c.close()
            clients.clear()
        server.close()

if __name__ == "__main__":
    start_server()


[SERVER AKTIF] Menunggu koneksi di 127.0.0.1:8080
[TERHUBUNG] ('127.0.0.1', 55505)
